## Upload personal data to database
## ( the best version )

#### Import the libraries to get the env variables

In [1]:
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

OPENAI_API_TYPE = "azure"
OPENAI_API_VERSION = os.environ.get("OPENAI_API_VERSION")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_API_BASE = os.environ.get("OPENAI_API_BASE")
OPENAI_API_VERSION = "2023-03-15-preview"

In [3]:
OPENAI_API_KEY 

'9ac347d13e834f288a2076ff9c7b418a'

#### Set the name of the database

In [4]:
database = "./football_documents/"

#### Import the libraries to get the text and split it in chunks

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader, DirectoryLoader

##### Get the text from the documents

In [6]:
loader = DirectoryLoader(database, glob="./*.txt", loader_cls=TextLoader)

documents = loader.load()
documents


[Document(page_content="Cristiano Ronaldo is undeniably one of the greatest footballers of all time, with a career that has spanned over two decades and has left an indelible mark on the sport. Born on February 5, 1985, in Funchal, Madeira, Portugal, Ronaldo's journey to stardom is a remarkable story of dedication, talent, and relentless ambition.\n\n**Early Life and Beginnings:**\nRonaldo's love for football began at a young age. Growing up in a modest family, he honed his skills playing on the streets of Madeira. At the tender age of 12, he moved to mainland Portugal to join the Sporting CP youth academy. His talent was evident from the start, and it didn't take long for him to break into the first team. His performances in the Primeira Liga caught the attention of scouts from top European clubs.\n\n**Rise to Prominence:**\nIn 2003, Ronaldo made a high-profile move to Manchester United for a then-record fee for a teenager. Under the guidance of Sir Alex Ferguson, he blossomed into a 

##### Split the text into chunks

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

docs

[Document(page_content="Cristiano Ronaldo is undeniably one of the greatest footballers of all time, with a career that has spanned over two decades and has left an indelible mark on the sport. Born on February 5, 1985, in Funchal, Madeira, Portugal, Ronaldo's journey to stardom is a remarkable story of dedication, talent, and relentless ambition.\n\n**Early Life and Beginnings:**\nRonaldo's love for football began at a young age. Growing up in a modest family, he honed his skills playing on the streets of Madeira. At the tender age of 12, he moved to mainland Portugal to join the Sporting CP youth academy. His talent was evident from the start, and it didn't take long for him to break into the first team. His performances in the Primeira Liga caught the attention of scouts from top European clubs.", metadata={'source': 'football_documents\\cristiano.txt'}),
 Document(page_content="**Rise to Prominence:**\nIn 2003, Ronaldo made a high-profile move to Manchester United for a then-record

#### Import the libraries to connect to Chroma database

In [8]:
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

In [9]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


##### Upload the text in chroma database

In [10]:
import re

text = "football_documents"
result = re.search(r'(?P<word>\w+)_', text)

if result:
    extracted_word = result.group('word')
    print(extracted_word)

database_name = extracted_word+"-database"
database_name

football


'football-database'

In [11]:
db2 = Chroma.from_documents(docs, embedding_function, persist_directory=database_name)